# Homework 1

In [1]:
# import libraries
import pyarrow

import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

In [2]:
pd.__version__

'2.2.3'

In [3]:
# load data
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
#Question 1: How many columns are in the df from January
print("The shape of the January data read is: ", df1.shape, 'and we have', df1.shape[1], 'columns.')

The shape of the January data read is:  (3066766, 19) and we have 19 columns.


In [5]:
df1.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
#Question 2: Std in minutes of the duration of trips in January
df1['duration'] = df1.tpep_dropoff_datetime - df1.tpep_pickup_datetime
df1.duration = df1.duration.apply(lambda td: td.total_seconds()/60)

print("The average std of the trips in january is", df1.duration.std())

The average std of the trips in january is 42.59435124195458


In [7]:
before = df1.shape[0]

In [8]:
#Question 3: Remove outliers (duration between 1 and 60 inclusive). What percentage of records was left?
df1 = df1[(df1.duration >= 1) & (df1.duration <= 60)]
print("The percentage of records we are left with corresponds to", df1.shape[0]*100/before, "%.")

The percentage of records we are left with corresponds to 98.1220282212598 %.


In [9]:
#######
df1.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [10]:
df1['PULocationID'] = df1['PULocationID'].astype(str)
df1['DOLocationID'] = df1['DOLocationID'].astype(str)
c = ['PULocationID','DOLocationID']
dicts = df1[c].to_dict(orient='records') 
dv = DictVectorizer()
X = dv.fit_transform(dicts)

print(X.shape)

(3009173, 515)


In [11]:
#######

In [12]:
y = df1['duration'].values

In [13]:
lr = LinearRegression()
lr.fit(X, y)

LinearRegression()

In [14]:
#rmse of lr on the training data

In [15]:
y_pred = lr.predict(X)
root_mean_squared_error(y, y_pred)

7.6492624397080675

In [16]:
######

In [17]:
len(df1), len(df2)

(3009173, 2913955)

In [20]:
#transformations in df2
df2['duration'] = df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime
df2.duration = df2.duration.apply(lambda td: td.total_seconds()/60)

df2 = df2[(df2.duration >= 1) & (df2.duration <= 60)]

df2['PULocationID'] = df2['PULocationID'].astype(str)
df2['DOLocationID'] = df2['DOLocationID'].astype(str)

/tmp/ipykernel_41792/2528267264.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['PULocationID'] = df2['PULocationID'].astype(str)
/tmp/ipykernel_41792/2528267264.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DOLocationID'] = df2['DOLocationID'].astype(str)


In [21]:
val_dicts = df2[c].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [22]:
target = 'duration'
y_train = df1[target].values
y_val = df2[target].values

In [23]:
lr = LinearRegression()
lr.fit(X, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.81181211389241

Answers:
19, 42.59, 98%, 515, 7.64